In [ ]:
%%capture
!pip install -q accelerate bitsandbytes
!pip install langchain==0.2.9
!pip install --upgrade langchain
!pip install -U langchain-community
!pip install typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install pydantic -U
!pip install pydantic==1.10.11
!pip install pypdf
!pip install sentence-transformers
!pip install chromadb
!pip install -U langchain-huggingface
!pip install huggingface_hub
!pip install accelerate
!pip install faiss-cpu


In [ ]:
!pip install -U langchain-community
#!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import InferenceClient
from langchain_core.messages import HumanMessage


In [ ]:
import os
from huggingface_hub import HfFolder
os.environ["HUGGINGFACE_TOKEN"] = "Your-HuggingFace-Token"
HfFolder.save_token(os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
import os
print(os.getcwd())

In [ ]:
def load_documents():
  document_loader = PyPDFLoader("/Users/srinandanasarmakesapragada/LangChain/The_last_men_standing.pdf")
  return document_loader.load()

In [ ]:
pdf_documents = load_documents()
print(pdf_documents[0].page_content)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
# text_splitter.split_documents(documents)[:5]
documents = text_splitter.split_documents(pdf_documents)

In [ ]:
!pip install sentencepiece


In [ ]:
repo_id = "HuggingFaceH4/zephyr-7b-beta"
model = HuggingFaceEndpoint(repo_id=repo_id, temperature=0.7)

In [ ]:
model.invoke("what is the meaning of life?")

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Muscle cars are the best",question="what are the best cars?")

In [ ]:
chain = prompt | model
chain.invoke({
    "context": "Aston Martin Valkyrie is the best-looking car. Its original name is Nebula, and the final codename is AM-RB 001. It is a sports car with a 2-door coupe and a 2-door targa top. Engine specifications: 6.5 L Aston Martin-Cosworth RA naturally-aspirated V12 with a 1,160 hp power output.",
    "question": "What is the best-looking car? What are the engine specifications?"
})

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='thenlper/gte-base')

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
os.environ["PINECONE_API_KEY"] = "Your-API"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

pinecone_client = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"],
    environment=os.environ["PINECONE_ENVIRONMENT"],
)

In [ ]:
index_name = "youtube-rag-index"
pinecone = PineconeVectorStore.from_documents(
    documents,embeddings, index_name=index_name
)

In [ ]:
pinecone.similarity_search("Who wrote THE LAST MEN STANDING")[:3]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
chain = (
    {"context" : pinecone.as_retriever(), "question":RunnablePassthrough()}
    |prompt
    |model
)

In [ ]:
chain.invoke("Who is Akira?")